### ***Problem02***
---

In [ ]:
!pip install gensim
!pip install sumy

In [39]:
### Import library & data set
# library
from google.colab import drive
drive.mount("/content/drive")
import os
import re
import time
import sys
import nltk
import numpy as np
import pandas as pd
from itertools import pairwise
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF
from sklearn.datasets import load_digits
from keras.preprocessing.text import Tokenizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import nltk
# summarization
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.kl import KLSummarizer
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
# dataset
# 20ng
categories = ['alt.atheism', 'talk.religion.misc','comp.graphics', 'sci.space']
ng = fetch_20newsgroups(categories=categories)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(ng.data)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
### 20ng dataset
text = []
for i in range(len(ng.data)):
  word = ng.data[i]
  word = re.sub(r"[^a-zA-Z]", " ", word)
  words = re.sub(r"\s+", " ", word)
  text.append(words)
df = pd.DataFrame(text, columns=["text"])

# normalization
df["text"] = df.text.str.strip()
df["text"] = df.text.str.lower()
df_20ng = np.array(df["text"])

In [60]:
### duc dataset
# read duc
path = "/content/drive/MyDrive/DS5230_Unsupervised Machine Learning/Assignments/HW05A/Summaries"
def get_txt_files_in_folder(folder_path, file_extension=".txt"):
    file_list = []
    for file in os.listdir(folder_path):
        if file.endswith(file_extension):
            file_list.append(os.path.join(folder_path, file))
    return file_list
duc = get_txt_files_in_folder(path)
duc = pd.DataFrame(duc)

# read txt
df = []
def read_text(path):
  arr = []
  path = str(np.array(path)).replace("[","").replace("]","").replace("'","")
  with open(path, 'r+', newline="", encoding="utf8") as document:
      for line in document:
        arr.append(line)
  return arr

for i in duc.loc[303]:
    data = read_text(i)
    df.append(data)
df = np.concatenate(df).tolist()

# normalization
text = []
for i in range(len(df)):
  word = ng.data[i]
  word = re.sub(r"[^a-zA-Z]", " ", word)
  words = re.sub(r"\s+", " ", word)
  text.append(words)
df = pd.DataFrame(text, columns=["topics"])

# normalization
df["topics"] = df.topics.str.strip()
df["topics"] = df.topics.str.lower()
df_duc = np.array(df["topics"])
df_duc = vectorizer.fit_transform(df_duc)

### ***20 news groups dataset***

In [42]:
### kl_sum in 20 news group
# hyperparamteres
LANGUAGE = "english"
SENTENCES_COUNT = 4

# model
stemmer = Stemmer(LANGUAGE)
kl = KLSummarizer(stemmer)
kl.stop_words = get_stop_words(LANGUAGE)

# parser
def parsing(text):
  parser = PlaintextParser.from_string(text,Tokenizer("english"))
  return parser.document

# implement
summaries = []
for i in df_20ng:
  sentence = kl(parsing(i), SENTENCES_COUNT)
  summaries.append(sentence)
result = pd.DataFrame(summaries, columns=["sentence"])
result.head()

,sentence
0,from rych festival ed ac uk r hawkes subject d...
1,subject re biblical backing of koresh s tape c...
2,from mark perew p f n z fidonet org subject re...
3,from dpw sei cmu edu david wood subject reques...
4,from prb access digex com pat subject conferen...
...,...
2029,from arthurc sfsuvax sfsu edu arthur chandler ...
2030,from schaefer sal sun usc edu peter schaefer s...
2031,from jamie genesis mcs com james r groves subj...
2032,from halat pooh bears jim halat subject islam ...


In [54]:
### kl_sum based on 20 news group with LDA topic modeling
# lda_topics=50
n_topics = 50
lda_50 = LatentDirichletAllocation(n_components=n_topics,random_state=22).fit(X)
components_50 = pd.DataFrame(lda_50.components_)

# max_topic
topics = []
max_columns = components_50.idxmax(axis=1)

# get_topic
X_token = vectorizer.get_feature_names_out()
for i in max_columns:
  word = X_token[i]
  topics.append(word)
topics = " ".join(topics)

# model
parser = PlaintextParser.from_string(topics,Tokenizer("english"))
stemmer = Stemmer(LANGUAGE)
kl = KLSummarizer(stemmer)
kl.stop_words = get_stop_words(LANGUAGE)

# implement
summary = kl(parser.document, 4)
summary

(<Sentence: buffalo rockets se ryukoku bolson diablo compaq ilmenau uwo virginia sandvik bham the convex comet nasa uiuc ncsu tek gamma ucsc orion freed objective duke tammy alaska joachim dreams liar nickh koresh fonts lippard shafer watson mccall schaefer format sysmgr incredulity higgins msu photoshop camosun bake mom umich crchh410 brothers>,)

### ***duc 2001 dataset***


In [56]:
### kl_sum in duc 2001
# hyperparamteres
LANGUAGE = "english"
SENTENCES_COUNT = 4

# model
stemmer = Stemmer(LANGUAGE)
kl = KLSummarizer(stemmer)
kl.stop_words = get_stop_words(LANGUAGE)

# parser
def parsing(text):
  parser = PlaintextParser.from_string(text,Tokenizer("english"))
  return parser.document

# implement
summaries = []
for i in df_duc:
  sentence = kl(parsing(i), SENTENCES_COUNT)
  summaries.append(sentence)
result = pd.DataFrame(summaries, columns=["sentence"])
result.head()

,sentence
0,from rych festival ed ac uk r hawkes subject d...
1,subject re biblical backing of koresh s tape c...
2,from mark perew p f n z fidonet org subject re...
3,from dpw sei cmu edu david wood subject reques...
4,from prb access digex com pat subject conferen...
5,from nanci ann miller nm w andrew cmu edu subj...
6,from wales larrison ofa fidonet org subject re...
7,from mjw cl cam ac uk m j williams subject re ...
8,from henry zoo toronto edu henry spencer subje...
9,from hendrix oasys dt navy mil dane hendrix su...


In [61]:
### kl_sum based on 20 news group with LDA topic modeling
# lda_topics=50
n_topics = 50
lda_50 = LatentDirichletAllocation(n_components=n_topics,random_state=22).fit(df_duc)
components_50 = pd.DataFrame(lda_50.components_)

# max_topic
topics = []
max_columns = components_50.idxmax(axis=1)

# get_topic
X_token = vectorizer.get_feature_names_out()
for i in max_columns:
  word = X_token[i]
  topics.append(word)
topics = " ".join(topics)

# model
parser = PlaintextParser.from_string(topics,Tokenizer("english"))
stemmer = Stemmer(LANGUAGE)
kl = KLSummarizer(stemmer)
kl.stop_words = get_stop_words(LANGUAGE)

# implement
summary = kl(parser.document, 4)
summary

(<Sentence: kewageshig almanac abc arm the challenges rot tony observations file halat beauchaine abc software abc brody ft clementine list maddi abc comet manned abc abc abc murder cc abc abc henry abc allen your abc liar abc com abc abc sq abc abc abc orbit christian abc gov abc cix>,)